In [1]:
import os, glob
import tensorflow as tf
import numpy as np
DATA_FOLDER = "/n/sd7/trung/csp/data/swbd"

/home/trungdv/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [26]:
with open(os.path.join(DATA_FOLDER, "vocab", "word_freq5_300h.txt")) as f:
    words = f.read().split('\n')
    word_dict = {word: i for i, word in enumerate(words)}
print("Vocab Size:", len(words))

Vocab Size: 11179


In [39]:
dlgs_swda = []
for file in glob.glob(os.path.join(DATA_FOLDER, "swda", '**/*.csv')):
    dlgid = os.path.basename(file).split('.')[0].split('_')[-1]
    dlgs_swda.append(dlgid)

print("swda conversation:", len(dlgs_swda))

swda conversation: 1155


In [57]:
dlgs = []
for file in glob.glob(os.path.join(DATA_FOLDER, "swb_ms98_transcriptions",  '**/**')):
    dlgid = os.path.basename(file)
    dlgs.append(dlgid)

dlgs_test = []
dlgs_train = dlgs
#dlgs_test = dlgs[:20]
#dlgs_train = dlgs[20:]
print("full dataset:", len(dlgs))

full dataset: 2438


In [56]:
with open(os.path.join(DATA_FOLDER, "train", "word_freq5_start_frame.csv")) as f:
    lines = f.read().split('\n')[1:]
    lines = [line.split(',') for line in lines]
    lines = [(line[2], line[3], line[4]) for line in lines if len(line) == 5]
    
print("Train Size:", len(lines))

Train Size: 278040


In [53]:
#eval2000
with open(os.path.join(DATA_FOLDER, "test", "word_freq5_start_frame.csv")) as f:
    lines = f.read().split('\n')[1:]
    lines = [line.split(',') for line in lines if line != '']
    lines_test = []
    for line in lines:
        s = line[4]
        s = s.replace('do not', "don't").replace('it is', "it's").replace('cannot', "can't")
        s = s.split('_')
        s = list(filter(lambda x: x[0] != '%', s))
        s = [word_dict[word] if word in word_dict else word_dict['OOV'] for word in s]
        s = ' '.join([str(x) for x in s])
        if len(s) > 0:
            lines_test.append((line[2], line[3], s))
    
print("Test Size:", len(lines_test))
#print('\n'.join([str(line[2]) for line in test_lines]))

Test Size: 1782


In [22]:
# baseline: eval2000
dlgs_test = []
dlgs_train = dlgs
bucket_size = 3000

mode = "train"
if False:
    _dlgs = dlgs_train if mode == "train" else dlgs_test
    _lines = list(filter(lambda it: os.path.basename(it[0])[2:6] in dlgs, lines))
    _lines = lines
    #print(len(lines_train))
    outputs = ["%s\t%s" % (line[0], line[2]) for line in _lines]
    #if mode == "train":
    #    outputs.sort(key=lambda s: len(s))
    
    '''
    writer = None
    for i, line in enumerate(lines_train):
        if mode == "test" and i == 0:
            writer = tf.python_io.TFRecordWriter(os.path.join(DATA_FOLDER, mode, "baseline_swda", "%s.tfrecords" % (mode)))
        if mode == "train" and i % bucket_size == 0:
            if writer is not None: writer.close()
            train_filename = os.path.join(DATA_FOLDER, mode, "baseline_swda", "%s_%d.tfrecords" % (mode, i // bucket_size + 1))
            writer = tf.python_io.TFRecordWriter(train_filename)
            print(i, end=" ")
        inp = np.load(line[0])
        inp = np.ndarray.flatten(inp)
        feature = {
            "input": tf.train.Feature(float_list=tf.train.FloatList(value=inp)),
            "target": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(x) for x in line[2].split(' ')]))
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())
    '''
    
    with open(os.path.join(DATA_FOLDER, "inputs_baseline_freq5_%s.txt" % mode), "w") as f:
        f.write('\n'.join(outputs))
        
if True:
    outputs = ["%s\t%s" % (line[0], line[2]) for line in lines_test]
    with open(os.path.join(DATA_FOLDER, "inputs_baseline_freq5_%s.txt" % "test"), "w") as f:
        f.write('\n'.join(outputs))

In [59]:
# baseline no 2000
dlgs_test = dlgs_swda[:15]
dlgs_train = dlgs_swda[15:]
mode = "train"
for mode in ["train", "test"]:
    _dlgs = dlgs_train if mode == "train" else dlgs_test
    _lines = list(filter(lambda it: os.path.basename(it[0])[2:6] in _dlgs, lines))
    #lines_train = lines
    print(len(_lines))
    outputs = ["%s\t%s" % (line[0], line[2]) for line in _lines]
    #if mode == "train":
    #    outputs.sort(key=lambda s: len(s))
    
    '''
    writer = None
    for i, line in enumerate(lines_train):
        if mode == "test" and i == 0:
            writer = tf.python_io.TFRecordWriter(os.path.join(DATA_FOLDER, mode, "baseline_swda", "%s.tfrecords" % (mode)))
        if mode == "train" and i % bucket_size == 0:
            if writer is not None: writer.close()
            train_filename = os.path.join(DATA_FOLDER, mode, "baseline_swda", "%s_%d.tfrecords" % (mode, i // bucket_size + 1))
            writer = tf.python_io.TFRecordWriter(train_filename)
            print(i, end=" ")
        inp = np.load(line[0])
        inp = np.ndarray.flatten(inp)
        feature = {
            "input": tf.train.Feature(float_list=tf.train.FloatList(value=inp)),
            "target": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(x) for x in line[2].split(' ')]))
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())
    '''
    
    with open(os.path.join(DATA_FOLDER, "inputs_baseline_freq5_inclusive_eval_da_%s.txt" % mode), "w") as f:
        f.write('\n'.join(outputs))

133117
1980


In [17]:
mode = "train"
if True:
    dlgs = dlgs_train if mode == "train" else dlgs_test
    #lines_train = list(filter(lambda it: os.path.basename(it[0])[2:6] in dlgs, lines))
    lines_train = lines
    print(len(lines_train))
    outputs = ["%s\t%s\t%s" % (line[0], line[1], line[2]) for line in lines_train]
    #if mode == "train":
    #    outputs.sort(key=lambda s: len(s))
    
    '''
    writer = None
    for i, line in enumerate(lines_train):
        if mode == "test" and i == 0:
            writer = tf.python_io.TFRecordWriter(os.path.join(DATA_FOLDER, mode, "baseline_swda", "%s.tfrecords" % (mode)))
        if mode == "train" and i % bucket_size == 0:
            if writer is not None: writer.close()
            train_filename = os.path.join(DATA_FOLDER, mode, "baseline_swda", "%s_%d.tfrecords" % (mode, i // bucket_size + 1))
            writer = tf.python_io.TFRecordWriter(train_filename)
            print(i, end=" ")
        inp = np.load(line[0])
        inp = np.ndarray.flatten(inp)
        feature = {
            "input": tf.train.Feature(float_list=tf.train.FloatList(value=inp)),
            "target": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(x) for x in line[2].split(' ')]))
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())
    '''
    
    with open(os.path.join(DATA_FOLDER, "inputs_start_frame_freq5_%s.txt" % mode), "w") as f:
        f.write('\n'.join(outputs))
        
if True:
    outputs = ["%s\t%s\t%s" % (line[0], line[1], line[2]) for line in lines_test]
    with open(os.path.join(DATA_FOLDER, "inputs_start_frame_freq5_%s.txt" % "test"), "w") as f:
        f.write('\n'.join(outputs))

278041


In [23]:
for mode in ["train", "test"]:
    dlgs = dlgs_train if mode == "train" else dlgs_test
    lines_train = list(filter(lambda it: os.path.basename(it[0])[2:6] in dlgs, lines))
    print(len(lines_train))
    outputs = ["%s\t%s\t%s" % (line[0], line[1], line[2]) for line in lines_train]
    
    
    for i, line in enumerate(lines_train):
        if i % bucket_size == 0:
            train_filename = os.path.join(DATA_FOLDER, "input_dlg_order_%s_%d.tfrecords" % (mode, i // bucket_size + 1))
            writer = tf.python_io.TFRecordWriter(train_filename)
            print(i, end=" ")
        inp = np.load(line[0])
        inp = np.ndarray.flatten(inp)
        feature = {
            "input": tf.train.Feature(float_list=tf.train.FloatList(value=inp)),
            "target": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(x) for x in line[2].split(' ')]))
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())
    
    with open(os.path.join(DATA_FOLDER, "inputs_dlg_order_%s.txt" % mode), "w") as f:
        f.write('\n'.join(outputs))

132364
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 

KeyboardInterrupt: 